In [1]:
import os

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from utils import tabularModel, titanicDataset

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch

import time
%matplotlib inline

In [2]:
train_data = pd.read_csv(os.path.join('data', 'titanic', 'test.csv'))

In [3]:
train_data.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [4]:
def train(config):
    batch_size = int(config['batch_size'])
    num_embedding = int(config['num_embedding'])
    learning_rate = config['learning_rate']
    fc_size = (int(config['fc_size_0']), int(config['fc_size_1']))
    dropout = (config['dropout_0'], config['dropout_1'])
    dataset = titanicDataset(os.path.join('/ldap_home', 'chang.liao', '4483A3', 'data', 'titanic', 'train.csv'))
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset,
                                                                 [int(0.8 * len(dataset)),
                                                                  len(dataset) - int(0.8 * len(dataset))])
    train_dataloader = DataLoader(train_dataset,
                                  batch_size=batch_size, shuffle=True, num_workers=4)
    valid_dataloader = DataLoader(valid_dataset,
                                 batch_size=len(valid_dataset), shuffle=False, num_workers=4)
    categorical_columns = ['Pclass', 'Sex', 'Embarked', 'binned_family']
    categories = [3, 2, 3, 4]
    continuous_columns = ['Fare', 'Age']
    embedding_size = list(zip(categories, [num_embedding] * 4))
    model = tabularModel(embedding_size, categorical_columns, continuous_columns, fc_size, dropout).double()
    optim = torch.optim.Adagrad(model.parameters(), lr=learning_rate, weight_decay=0.95)
    lossfunc = nn.CrossEntropyLoss()
    num_epoch = 300
    best_accu = 0
    cnt = 0
    model.train()
    for epoch in range(num_epoch):
        for i, (x, y) in enumerate(train_dataloader):
            optim.zero_grad()
            pred = model(x)
            loss = lossfunc(pred, y.squeeze().long())
            loss.backward()
            optim.step()

            if i % 10 == 0 or i == (len(train_dataset) / batch_size) - 1 :
                with torch.no_grad():
                    model.eval()
                    x, y = next(iter(valid_dataloader))
                    pred = model(x)
                    valid_loss = lossfunc(pred, y.squeeze().long())
                    accu = (pred.argmax(-1) == y.squeeze()).sum().item() / len(valid_dataset)
                # print("Epoch: {} | Train loss: {:.4f} | Valid loss: {:.4f} | Valid Accu: {:.4f}".format(
                #     epoch, loss.item(), valid_loss.item(), accu
                # ))
                model.train()
        # print("\n {:.4f} {:.4f} {} \n".format(1.02 * best_accu, accu, cnt))
        if accu >= best_accu:
            cnt = 0
            best_accu = accu
            # torch.save(model.state_dict(), os.path.join('model', 'torch_nn',
            #                                             'model-{}.pt'.format(epoch)))
        else:
            cnt += 1

        if cnt == 50:
            # print("Early stopped!")
            break
    return best_accu

In [5]:
def raytune_trainable(config, checkpoint_dir=None):
    tune.report(accu=train(config))

In [6]:
search_space = {
    'batch_size': (2, 64),
    'num_embedding': (4, 256),
    'learning_rate': (1e-4, 0.1),
    'fc_size_0': (64, 256),
    'fc_size_1': (8, 128),
    'dropout_0': (0.1, 0.8),
    'dropout_1': (0.1, 0.8)
}

In [10]:
algo = BayesOptSearch(space=search_space, metric='accu', mode='max', random_search_steps=20)
start = time.time()
analysis = tune.run(raytune_trainable, search_alg=algo, num_samples=500, 
                    # checkpoint_freq=4,
                    # checkpoint_at_end=True,
                    local_dir=os.curdir,
                    resources_per_trial={'cpu': 9},
                    scheduler = ASHAScheduler(
                        metric='accu',
                        mode='max'),
                    raise_on_failed_trial=False
                   )
end = time.time()
print(end-start)

2020-11-10 23:28:12,512	WARNING tune.py:401 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding
raytune_trainable_5850786a,RUNNING,,25.2215,0.7655,0.612396,178.942,26.7222,0.0156839,18.6371


Result for raytune_trainable_5850786a:
  accu: 0.8435754189944135
  date: 2020-11-10_23-28-39
  done: false
  experiment_id: c84efc176d944e298dc07eff3f7c12f5
  experiment_tag: 1_batch_size=25.221,dropout_0=0.7655,dropout_1=0.6124,fc_size_0=178.94,fc_size_1=26.722,learning_rate=0.015684,num_embedding=18.637
  hostname: shopee-ds-ip-10-128-148-194.idata-server.shopee.io
  iterations_since_restore: 1
  node_ip: 10.128.148.194
  pid: 23740
  time_since_restore: 25.166114807128906
  time_this_iter_s: 25.166114807128906
  time_total_s: 25.166114807128906
  timestamp: 1605022119
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5850786a
  


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_5850786a,RUNNING,10.128.148.194:23740,25.2215,0.7655,0.612396,178.942,26.7222,0.0156839,18.6371,1,25.1661,0.843575
raytune_trainable_5852f9be,RUNNING,,55.7029,0.520781,0.595651,67.9522,124.389,0.083261,57.5095,,,
raytune_trainable_58544e2c,RUNNING,,13.2731,0.228383,0.31297,164.753,59.8334,0.0291938,158.187,,,
raytune_trainable_58553d3c,RUNNING,,10.6486,0.304501,0.356453,151.565,102.221,0.0200474,133.587,,,
raytune_trainable_58562daa,RUNNING,,38.7297,0.132515,0.525281,96.7406,15.8062,0.0948937,247.339,,,
raytune_trainable_585712a6,RUNNING,,52.1206,0.31323,0.16837,195.373,60.8183,0.0122916,128.785,,,
raytune_trainable_5857f482,RUNNING,,4.13209,0.736524,0.281146,191.204,45.4053,0.0520548,141.771,,,
raytune_trainable_5858dfe6,PENDING,,13.461,0.778709,0.642593,244.384,115.379,0.0598302,236.312,,,


Result for raytune_trainable_58544e2c:
  accu: 0.8212290502793296
  date: 2020-11-10_23-29-10
  done: true
  experiment_id: b29754302c3949899935ad2fa15ad361
  experiment_tag: 3_batch_size=13.273,dropout_0=0.22838,dropout_1=0.31297,fc_size_0=164.75,fc_size_1=59.833,learning_rate=0.029194,num_embedding=158.19
  hostname: shopee-ds-ip-10-128-148-194.idata-server.shopee.io
  iterations_since_restore: 1
  node_ip: 10.128.148.194
  pid: 23741
  time_since_restore: 56.08695960044861
  time_this_iter_s: 56.08695960044861
  time_total_s: 56.08695960044861
  timestamp: 1605022150
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 58544e2c
  


...

Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_147e662a,RUNNING,,18.6856,0.162648,0.234227,242.455,108.207,0.0590236,174.184,,,
raytune_trainable_26f3bfda,RUNNING,10.128.148.194:14594,36.8128,0.497055,0.410817,84.1513,116.46,0.059737,195.898,1,21.3475,0.843575
raytune_trainable_5af05b2a,RUNNING,,2.64433,0.566801,0.792588,128.922,67.949,0.071734,27.9849,,,
raytune_trainable_bff40b14,RUNNING,,2.98472,0.270448,0.28036,169.291,100.241,0.0778438,78.0306,,,
raytune_trainable_cfb164b6,RUNNING,,5.8212,0.695881,0.396833,196.326,92.9745,0.0874824,189.157,,,
raytune_trainable_e05a7e10,RUNNING,,15.2467,0.570487,0.692649,92.6121,120.177,0.0642094,75.8691,,,
raytune_trainable_f19b10f4,RUNNING,,13.4282,0.472825,0.746019,205.183,103.034,0.00200669,96.95,,,
raytune_trainable_365ab032,PENDING,,29.3337,0.791008,0.499816,109.929,94.3959,0.0991929,148.055,,,
raytune_trainable_0081a350,TERMINATED,,27.5084,0.777045,0.283553,89.1599,8.34247,0.094363,138.593,1,196.863,0.592179
raytune_trainable_038c64f8,TERMINATED,,2.14535,0.436243,0.181375,237.006,62.1063,0.033133,192.972,1,405.934,0.620112


Result for raytune_trainable_f19b10f4:
  accu: 0.888268156424581
  date: 2020-11-11_00-32-53
  done: false
  experiment_id: 4648cacbda2e4447bd5b515222e6419e
  experiment_tag: 115_batch_size=13.428,dropout_0=0.47282,dropout_1=0.74602,fc_size_0=205.18,fc_size_1=103.03,learning_rate=0.0020067,num_embedding=96.95
  hostname: shopee-ds-ip-10-128-148-194.idata-server.shopee.io
  iterations_since_restore: 1
  node_ip: 10.128.148.194
  pid: 14404
  time_since_restore: 128.74405002593994
  time_this_iter_s: 128.74405002593994
  time_total_s: 128.74405002593994
  timestamp: 1605025973
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: f19b10f4
  


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_147e662a,RUNNING,,18.6856,0.162648,0.234227,242.455,108.207,0.0590236,174.184,,,
raytune_trainable_365ab032,RUNNING,,29.3337,0.791008,0.499816,109.929,94.3959,0.0991929,148.055,,,
raytune_trainable_5af05b2a,RUNNING,,2.64433,0.566801,0.792588,128.922,67.949,0.071734,27.9849,,,
raytune_trainable_bff40b14,RUNNING,,2.98472,0.270448,0.28036,169.291,100.241,0.0778438,78.0306,,,
raytune_trainable_cfb164b6,RUNNING,,5.8212,0.695881,0.396833,196.326,92.9745,0.0874824,189.157,,,
raytune_trainable_e05a7e10,RUNNING,,15.2467,0.570487,0.692649,92.6121,120.177,0.0642094,75.8691,,,
raytune_trainable_f19b10f4,RUNNING,10.128.148.194:14404,13.4282,0.472825,0.746019,205.183,103.034,0.00200669,96.95,1,128.744,0.888268
raytune_trainable_43e874b4,PENDING,,36.6302,0.723612,0.500065,141.721,115.888,0.0837299,133.054,,,
raytune_trainable_0081a350,TERMINATED,,27.5084,0.777045,0.283553,89.1599,8.34247,0.094363,138.593,1,196.863,0.592179
raytune_trainable_038c64f8,TERMINATED,,2.14535,0.436243,0.181375,237.006,62.1063,0.033133,192.972,1,405.934,0.620112


Result for raytune_trainable_147e662a:
  accu: 0.776536312849162
  date: 2020-11-11_00-33-16
  done: true
  experiment_id: 867298160fcc4663ba499c3f419ff3a4
  experiment_tag: 118_batch_size=18.686,dropout_0=0.16265,dropout_1=0.23423,fc_size_0=242.46,fc_size_1=108.21,learning_rate=0.059024,num_embedding=174.18
  hostname: shopee-ds-ip-10-128-148-194.idata-server.shopee.io
  iterations_since_restore: 1
  node_ip: 10.128.148.194
  pid: 14618
  time_since_restore: 120.29243803024292
  time_this_iter_s: 120.29243803024292
  time_total_s: 120.29243803024292
  timestamp: 1605025996
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 147e662a
  


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_365ab032,RUNNING,,29.3337,0.791008,0.499816,109.929,94.3959,0.0991929,148.055,,,
raytune_trainable_43e874b4,RUNNING,,36.6302,0.723612,0.500065,141.721,115.888,0.0837299,133.054,,,
raytune_trainable_5af05b2a,RUNNING,,2.64433,0.566801,0.792588,128.922,67.949,0.071734,27.9849,,,
raytune_trainable_bff40b14,RUNNING,,2.98472,0.270448,0.28036,169.291,100.241,0.0778438,78.0306,,,
raytune_trainable_cfb164b6,RUNNING,,5.8212,0.695881,0.396833,196.326,92.9745,0.0874824,189.157,,,
raytune_trainable_e05a7e10,RUNNING,,15.2467,0.570487,0.692649,92.6121,120.177,0.0642094,75.8691,,,
raytune_trainable_62045fb2,PENDING,,21.927,0.499531,0.629364,217.041,9.55955,0.0985924,112.383,,,
raytune_trainable_0081a350,TERMINATED,,27.5084,0.777045,0.283553,89.1599,8.34247,0.094363,138.593,1,196.863,0.592179
raytune_trainable_038c64f8,TERMINATED,,2.14535,0.436243,0.181375,237.006,62.1063,0.033133,192.972,1,405.934,0.620112
raytune_trainable_0ab12c62,TERMINATED,,30.1184,0.443581,0.683732,226.991,51.151,0.0188962,223.671,1,46.5032,0.815642


In [9]:
analysis.best_result

NameError: name 'analysis' is not defined